# How to complete this assignment
First, download [this Kaggle dataset](https://www.kaggle.com/hugomathien/soccer) and extract *sqlite* database. You may need to register at https://www.kaggle.com/ beforehand. Then complete 15 graded tasks below, the score is given in brackets. Finally submit the resulting `.ipynb` file to rs-app Auto-test.

- Do not delete or rename the variables given before the inscription `#your code here`, they are needed for the correct verification.
- Do not change the code in the last Notebook cell, it is required for the server check.
- Your Notebook must run completely without errors to be graded! Please check everything before submission by going *Cell -> Run All*

## Some important notes
- If you need to **calculate the number of "something"** that means we expect you to assign an Integer to the given variable
- If you need to **make a list of "something"** we expect you to assign a Python list with appropriate values to the given variable
- If you need to find a **specifiс player, day of the week, team, etc.** we expect you to assign a String with the full name of the entity to the given variable (`player_name`, day of week full name, `team_name`, etc.)

In [106]:
import sqlite3
import pandas as pd
import os
import numpy as np

pd.set_option('display.max_column', None)

In [2]:
# Leave that code unchanged, it is required for the server check!
db = sqlite3.connect(os.environ.get("DB_PATH") or 'database.sqlite')

In [3]:
# You may load the data from SQL table directly to the Pandas dataframe as
player_data = pd.read_sql("SELECT * FROM Player;", db)

In [4]:
player_data.head()

,id,player_api_id,player_name,player_fifa_api_id,birthday,height,weight
0,1,505942,Aaron Appindangoye,218353,1992-02-29 00:00:00,182.88,187
1,2,155782,Aaron Cresswell,189615,1989-12-15 00:00:00,170.18,146
2,3,162549,Aaron Doran,186170,1991-05-13 00:00:00,170.18,163
3,4,30572,Aaron Galindo,140161,1982-05-08 00:00:00,182.88,198
4,5,23780,Aaron Hughes,17725,1979-11-08 00:00:00,182.88,154


**Task 1 (0.25 point).** Calculate the number of players with a height between 180 and 190 inclusive

In [5]:
players_180_190 = len(player_data[player_data['height'].between(180, 190)])# Your code here

In [7]:
players_180_190

5925

In [8]:
assert(isinstance(players_180_190, int))

**Task 2 (0.25 point).** Calculate the number of players born in 1980. <br>
**Hint:** you may want to cast your 'birthday' column to DateTime type by [pandas.to_datetime](https://pandas.pydata.org/docs/reference/api/pandas.to_datetime.html)

In [9]:
players_1980 = 0 # Your code here
for i, v in pd.to_datetime(player_data["birthday"]).items():
    if v.year == 1980:
        players_1980+=1

In [10]:
players_1980

434

In [74]:
assert(isinstance(players_1980, int))

**Task 3 (0.25 point).** Make a list of the top 10 players with the highest weight sorted in descending order. If there are several players with the same weight put them in the lexicographic order by name.

In [11]:
highest_players = player_data.sort_values(["weight", "player_name"], ascending = False).player_name[:10].tolist()# Your code here

In [12]:
highest_players

['Tim Wiese',
 'Kristof van Hout',
 'Jeroen Verhoeven',
 'Ishmael Miller',
 'Marcus Hahnemann',
 'Lars Unnerstall',
 'Christopher Samba',
 'Cassio',
 'Wojciech Kaczmarek',
 'Enoch Showunmi']

In [13]:
assert(len(highest_players) == 10)
assert(isinstance(highest_players, list))
for i in range(10):
    assert(isinstance(highest_players[i], str))

**Task 4 (0.5 point).** Make a list of tuples containing years along with the number of players born in that year from 1980 up to 1990. <br>
**Structure example**: [(1980, 123), (1981, 140) ..., (1990, 83)] -> There were born 123 players in 1980, there were born 140 players in 1981 and etc.

In [14]:
subset = pd.to_datetime(player_data["birthday"]).apply(lambda x: int(x.year) if (x.year >=1980) & (x.year <= 1990) else None).dropna()

In [15]:
years_born_players = [(int(idx), int(val)) for idx, val in subset.to_frame("years").groupby("years")["years"].count().items()]# Your code here

In [16]:
years_born_players

[(1980, 434),
 (1981, 475),
 (1982, 487),
 (1983, 541),
 (1984, 607),
 (1985, 593),
 (1986, 710),
 (1987, 722),
 (1988, 802),
 (1989, 723),
 (1990, 696)]

In [17]:
assert(len(years_born_players) == 11)
assert(isinstance(years_born_players, list))
for i in range(10):
    assert(isinstance(years_born_players[i], tuple))
    assert(isinstance(years_born_players[i][0], int))
    assert(isinstance(years_born_players[i][1], int))

**Task 5 (0.5 point).** Calculate the mean and the standard deviation of the players' **height** with the name **Adriano**. <br>
**Note:** Name is represented by the first part of `player_name`.

In [18]:
sub_data = pd.DataFrame()
sub_data = player_data["player_name"].str.split( n=1,expand=True)
sub_data["height"] = player_data["height"]

In [19]:
sub_data = sub_data.drop(sub_data[sub_data[0] != "Adriano"].index)

In [20]:
adriano_mean, adriano_std = sub_data["height"].mean(), sub_data["height"].std()# Your code here

In [21]:
adriano_mean, adriano_std

(182.118, 5.361479480723788)

In [22]:
assert(isinstance(adriano_mean, float))
assert(isinstance(adriano_std, float))

**Task 6 (0.75 point).** How many players were born on each day of the week? Find the day of the week with the minimum number of players born.

In [23]:
sub_data = pd.DataFrame()
sub_data["weekday"] = pd.to_datetime(player_data["birthday"]).apply(lambda x: x.strftime('%A'))

In [24]:
dow_with_min_players_born = sub_data.value_counts().idxmin()[0]# Your code here

In [25]:
dow_with_min_players_born

'Sunday'

In [26]:
assert(isinstance(dow_with_min_players_born, str))

**Task 7 (0.75 point).** Find a league with the most matches in total. If there are several leagues with the same amount of matches, take the first in the lexical order.

In [27]:
league_data = pd.read_sql("SELECT * FROM League;", db)

In [28]:
match_data = pd.read_sql("SELECT * FROM Match;", db)

In [30]:
league_data.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


In [31]:
match_data.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.4,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [32]:
sub_data = match_data["league_id"].value_counts().to_frame()

In [33]:
sub_data

,league_id
1729,3040
4769,3040
21518,3040
10257,3017
7809,2448
13274,2448
17642,2052
15722,1920
19694,1824
1,1728


In [34]:
sub_data.max()[0]

3040

In [35]:
ids = sub_data[sub_data["league_id"] == sub_data.max()[0]].index.tolist()

In [36]:
league_most_matches = sorted(league_data[league_data["id"].isin(ids)].name.tolist())[0] # Your code here

In [37]:
league_most_matches

'England Premier League'

In [290]:
assert(isinstance(league_most_matches, str))

**Task 8 (1.25 point).** Find a player who participated in the largest number of matches during the whole match history. Assign a `player_name` to the given variable

In [38]:
player_columns = match_data.columns[12:77].tolist()

In [39]:
list_pl = []
for i in player_columns:
    list_pl.append(match_data[i].tolist())

In [40]:
list_pl = sum(list_pl, [])

In [41]:
players = pd.DataFrame({"id": list_pl})

In [42]:
max_matches_player = player_data[player_data["id"] == players.dropna().value_counts().idxmax()[0]]["player_name"].item()# Your code here

In [43]:
max_matches_player

'Aaron Doran'

In [44]:
assert(isinstance(max_matches_player, str))

**Task 9 (1.5 point).** List top-5 tuples of most correlated **player's characteristics** in the descending order of the absolute [Pearson's coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) value.

**Note 1:** Players characteristics are all the columns in `Player_Attributes` table except `[id, player_fifa_api_id, player_api_id, date, preferred_foot, attacking_work_rate, defensive_work_rate]`). <br>
**Note 2:** Exclude duplicated pairs from the list. E.g. ('gk_handling', 'gk_reflexes') and ('gk_reflexes', 'gk_handling') are duplicates, leave just one of them in the resulting list.

**Hint:** You may use [dataframe.corr()](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.corr.html) for calculating pairwise Pearson correlation.

In [45]:
attributes_data = pd.read_sql("SELECT * FROM Player_Attributes;", db)

In [46]:
attributes_data.head()

,id,player_fifa_api_id,player_api_id,date,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,1,218353,505942,2016-02-18 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,2,218353,505942,2015-11-19 00:00:00,67.0,71.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,3,218353,505942,2015-09-21 00:00:00,62.0,66.0,right,medium,medium,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,4,218353,505942,2015-03-20 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,5,218353,505942,2007-02-22 00:00:00,61.0,65.0,right,medium,medium,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0


In [47]:
attributes_data.drop(['id', 'player_fifa_api_id', 'player_api_id', 'date', 'preferred_foot', 'attacking_work_rate', 'defensive_work_rate'], axis=1)

,overall_rating,potential,crossing,finishing,heading_accuracy,short_passing,volleys,dribbling,curve,free_kick_accuracy,long_passing,ball_control,acceleration,sprint_speed,agility,reactions,balance,shot_power,jumping,stamina,strength,long_shots,aggression,interceptions,positioning,vision,penalties,marking,standing_tackle,sliding_tackle,gk_diving,gk_handling,gk_kicking,gk_positioning,gk_reflexes
0,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
1,67.0,71.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,71.0,70.0,45.0,54.0,48.0,65.0,69.0,69.0,6.0,11.0,10.0,8.0,8.0
2,62.0,66.0,49.0,44.0,71.0,61.0,44.0,51.0,45.0,39.0,64.0,49.0,60.0,64.0,59.0,47.0,65.0,55.0,58.0,54.0,76.0,35.0,63.0,41.0,45.0,54.0,48.0,65.0,66.0,69.0,6.0,11.0,10.0,8.0,8.0
3,61.0,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
4,61.0,65.0,48.0,43.0,70.0,60.0,43.0,50.0,44.0,38.0,63.0,48.0,60.0,64.0,59.0,46.0,65.0,54.0,58.0,54.0,76.0,34.0,62.0,40.0,44.0,53.0,47.0,62.0,63.0,66.0,5.0,10.0,9.0,7.0,7.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
183973,83.0,85.0,84.0,77.0,59.0,89.0,77.0,84.0,86.0,78.0,84.0,85.0,66.0,72.0,77.0,86.0,73.0,76.0,58.0,72.0,67.0,81.0,56.0,78.0,86.0,88.0,83.0,22.0,31.0,30.0,9.0,20.0,84.0,20.0,20.0
183974,78.0,80.0,74.0,76.0,53.0,84.0,77.0,85.0,86.0,74.0,73.0,86.0,66.0,67.0,77.0,74.0,73.0,75.0,58.0,66.0,65.0,73.0,61.0,64.0,72.0,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183975,77.0,80.0,74.0,71.0,53.0,84.0,77.0,85.0,86.0,74.0,73.0,86.0,66.0,67.0,77.0,74.0,73.0,75.0,58.0,66.0,65.0,73.0,67.0,64.0,72.0,88.0,70.0,32.0,31.0,30.0,9.0,20.0,73.0,20.0,20.0
183976,78.0,81.0,74.0,64.0,57.0,86.0,77.0,87.0,86.0,73.0,73.0,91.0,61.0,60.0,77.0,69.0,73.0,72.0,58.0,67.0,59.0,78.0,63.0,63.0,68.0,88.0,53.0,28.0,32.0,30.0,9.0,20.0,73.0,20.0,20.0


In [48]:
top5 = attributes_data.corr().unstack().sort_values(ascending=False).drop_duplicates()[1:6]

/var/folders/7s/78tlgxm5249b69r8knz_d7qw0000gn/T/ipykernel_65717/1555772855.py:1: FutureWarning: The default value of numeric_only in DataFrame.corr is deprecated. In a future version, it will default to False. Select only valid columns or specify the value of numeric_only to silence this warning.
  top5 = attributes_data.corr().unstack().sort_values(ascending=False).drop_duplicates()[1:6]


In [49]:
top5.to_frame().iloc[:, :-1].to_records()

rec.array([('gk_reflexes', 'gk_positioning'),
           ('gk_reflexes', 'gk_handling'),
           ('gk_positioning', 'gk_handling'),
           ('standing_tackle', 'sliding_tackle'),
           ('marking', 'standing_tackle')],
          dtype=[('level_0', 'O'), ('level_1', 'O')])

In [50]:
top_correlated_features = list(top5.to_frame().iloc[:, :-1].to_records())# Your code here

In [51]:
top_correlated_features

[('gk_reflexes', 'gk_positioning'),
 ('gk_reflexes', 'gk_handling'),
 ('gk_positioning', 'gk_handling'),
 ('standing_tackle', 'sliding_tackle'),
 ('marking', 'standing_tackle')]

In [52]:
for i in range(0,len(top_correlated_features)): 
    top_correlated_features[i] = tuple(top_correlated_features[i])

In [53]:
top_correlated_features

[('gk_reflexes', 'gk_positioning'),
 ('gk_reflexes', 'gk_handling'),
 ('gk_positioning', 'gk_handling'),
 ('standing_tackle', 'sliding_tackle'),
 ('marking', 'standing_tackle')]

In [54]:
assert(len(top_correlated_features) == 5)
assert(isinstance(top_correlated_features, list))
for i in range(5):
    assert(isinstance(top_correlated_features[i], tuple))
    assert(isinstance(top_correlated_features[i][0], str))
    assert(isinstance(top_correlated_features[i][1], str))

**Task 10 (2 points).** Find top-5 most similar players to **Neymar** whose names are given. The similarity is measured as [Euclidean distance](https://en.wikipedia.org/wiki/Euclidean_distance) between vectors of players' characteristics (described in the task above). Put their names in a vector in ascending order by Euclidean distance and sorted by `player_name` if the distance is the same<br>
**Note 1:** There are many records for some players in the `Player_Attributes` table. You need to take the freshest data (characteristics with the most recent `date`). <br>
**Note 2:** Use pure values of the characteristics even if you are aware of such preprocessing technics as normalization. <br>
**Note 3:** Please avoid using any built-in methods for calculating the Euclidean distance between vectors, think about implementing your own.

In [55]:
list_atr = ['player_api_id','date','gk_reflexes', 'gk_positioning','gk_handling','standing_tackle', 'sliding_tackle','marking']
df_corr = attributes_data[list_atr]

In [56]:
df_corr.head()

,player_api_id,date,gk_reflexes,gk_positioning,gk_handling,standing_tackle,sliding_tackle,marking
0,505942,2016-02-18 00:00:00,8.0,8.0,11.0,69.0,69.0,65.0
1,505942,2015-11-19 00:00:00,8.0,8.0,11.0,69.0,69.0,65.0
2,505942,2015-09-21 00:00:00,8.0,8.0,11.0,66.0,69.0,65.0
3,505942,2015-03-20 00:00:00,7.0,7.0,10.0,63.0,66.0,62.0
4,505942,2007-02-22 00:00:00,7.0,7.0,10.0,63.0,66.0,62.0


In [57]:
id_naymar = player_data[player_data.apply(lambda row: row.astype(str).str.contains('Neymar').any(), axis=1)]['player_api_id']

In [58]:
id_naymar = id_naymar.values[0]

In [59]:
id_naymar

19533

In [60]:
neymar_atr = attributes_data[attributes_data['player_api_id'] == id_naymar].sort_values(by='date', ascending=False).iloc[0]

In [61]:
v_neymar = list(neymar_atr[list_atr][2:].values)

In [62]:
v_neymar

[11.0, 15.0, 9.0, 24.0, 33.0, 21.0]

In [63]:
test = df_corr.sort_values(by=['player_api_id','date'], ascending=False)

In [64]:
fresh_atr_data = test.drop_duplicates(subset=['player_api_id'], keep='first')

In [65]:
fresh_atr_data.head()

,player_api_id,date,gk_reflexes,gk_positioning,gk_handling,standing_tackle,sliding_tackle,marking
8085,750584,2016-05-19 00:00:00,13.0,11.0,9.0,66.0,62.0,56.0
147408,750435,2016-04-14 00:00:00,11.0,10.0,10.0,19.0,21.0,18.0
60179,748432,2016-05-12 00:00:00,6.0,12.0,8.0,69.0,68.0,63.0
58304,746419,2016-05-12 00:00:00,9.0,9.0,10.0,69.0,62.0,55.0
66601,744907,2016-06-09 00:00:00,9.0,12.0,14.0,49.0,50.0,40.0


In [66]:
list(fresh_atr_data.iloc[0, ])[2:]

[13.0, 11.0, 9.0, 66.0, 62.0, 56.0]

In [67]:
dist=[]
for i in range(0, len(fresh_atr_data)):
    s=0
    for j in  range(0, len(v_neymar)):
        s+=(v_neymar[j] - list(fresh_atr_data.iloc[i, ])[j+2])**2
    dist.append(s**0.5)

In [68]:
fresh_atr_data.insert(2, 'neymar_dist', dist)

In [69]:
ids_naymar_dist = list(fresh_atr_data.sort_values(by='neymar_dist')[1:6].player_api_id.values)

In [70]:
neymar_similarities = list(player_data[ player_data['player_api_id'].isin(ids_naymar_dist)].player_name.values)# Your code here

In [71]:
neymar_similarities

['Jaroslav Navratil',
 'Joshua John',
 'Milos Krstic',
 'Petar Sliskovic',
 'Thomas Guimaraes Azevedo']

In [72]:
assert(len(neymar_similarities) == 5)
assert(isinstance(neymar_similarities, list))
for i in range(5):
    assert(isinstance(neymar_similarities[i], str))

**Task 11 (1 point).** Calculate the number of home matches played by the **Borussia Dortmund** team in **Germany 1. Bundesliga** in season **2008/2009**

In [73]:
id_ger1 = league_data[league_data['name'] == 'Germany 1. Bundesliga']['id'].values[0]

In [74]:
id_ger1

7809

In [75]:
team_data = pd.read_sql("SELECT * FROM Team;", db)

In [76]:
team_data.head()

,id,team_api_id,team_fifa_api_id,team_long_name,team_short_name
0,1,9987,673.0,KRC Genk,GEN
1,2,9993,675.0,Beerschot AC,BAC
2,3,10000,15005.0,SV Zulte-Waregem,ZUL
3,4,9994,2007.0,Sporting Lokeren,LOK
4,5,9984,1750.0,KSV Cercle Brugge,CEB


In [77]:
id_borussia = team_data[team_data['team_long_name'] == 'Borussia Dortmund']['team_api_id'].values[0]

In [78]:
id_borussia

9789

In [79]:
match_data.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
0,1,1,1,2008/2009,1,2008-08-17 00:00:00,492473,9987,9993,1,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.73,3.40,5.00,1.75,3.35,4.20,1.85,3.2,3.5,1.80,3.3,3.75,NaN,NaN,NaN,1.70,3.30,4.33,1.90,3.3,4.00,1.65,3.40,4.50,1.78,3.25,4.00,1.73,3.40,4.20
1,2,1,1,2008/2009,1,2008-08-16 00:00:00,492474,10000,9994,0,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.95,3.20,3.60,1.80,3.30,3.95,1.90,3.2,3.5,1.90,3.2,3.50,NaN,NaN,NaN,1.83,3.30,3.60,1.95,3.3,3.80,2.00,3.25,3.25,1.85,3.25,3.75,1.91,3.25,3.60
2,3,1,1,2008/2009,1,2008-08-16 00:00:00,492475,9984,8635,0,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,2.38,3.30,2.75,2.40,3.30,2.55,2.60,3.1,2.3,2.50,3.2,2.50,NaN,NaN,NaN,2.50,3.25,2.40,2.63,3.3,2.50,2.35,3.25,2.65,2.50,3.20,2.50,2.30,3.20,2.75
3,4,1,1,2008/2009,1,2008-08-17 00:00:00,492476,9991,9998,5,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,1.44,3.75,7.50,1.40,4.00,6.80,1.40,3.9,6.0,1.44,3.6,6.50,NaN,NaN,NaN,1.44,3.75,6.00,1.44,4.0,7.50,1.45,3.75,6.50,1.50,3.75,5.50,1.44,3.75,6.50
4,5,1,1,2008/2009,1,2008-08-16 00:00:00,492477,7947,9985,1,3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None,None,None,None,None,None,None,5.00,3.50,1.65,5.00,3.50,1.60,4.00,3.3,1.7,4.00,3.4,1.72,NaN,NaN,NaN,4.20,3.40,1.70,4.50,3.5,1.73,4.50,3.40,1.65,4.50,3.50,1.65,4.75,3.30,1.67


In [80]:
borussia_bundesliga_2008_2009_matches = len(match_data[(match_data['league_id'] == id_ger1) & (match_data['season'] == '2008/2009') & (match_data['home_team_api_id'] == id_borussia)])# Your code here

In [81]:
borussia_bundesliga_2008_2009_matches

17

In [82]:
assert(isinstance(borussia_bundesliga_2008_2009_matches, int))

**Task 12 (1 point).** Find a team having the most matches (both home and away!) in the **Germany 1. Bundesliga** in **2008/2009** season. Return number of matches.

In [83]:
season_data = match_data[(match_data['league_id'] == id_ger1) & (match_data['season'] == '2008/2009')]

In [84]:
season_data = season_data[['home_team_api_id','away_team_api_id']]

In [85]:
list_home_team = list(season_data.home_team_api_id.values)
list_away_team = list(season_data.away_team_api_id.values)

In [86]:
list_teams = list_home_team + list_away_team

In [87]:
df_t = pd.DataFrame()

In [88]:
df_t["teams"] = list_teams

In [89]:
team_most_matches_bundesliga_2008_2009 = df_t.teams.value_counts().max()# Your code here

In [90]:
team_most_matches_bundesliga_2008_2009

34

In [91]:
assert(isinstance(team_most_matches_bundesliga_2008_2009, int))

**Task 13 (1 point).** Count total number of **Arsenal** matches (both home and away!) in the **2015/2016** season which they have won. <br><br>
**Note:** Winning a game means scoring **more** goals than an opponent.

In [92]:
id_arsenal = team_data[team_data['team_long_name'] == 'Arsenal']['team_api_id'].values[0]

In [93]:
arsenal_home = len(match_data[(match_data['home_team_api_id'] == id_arsenal)&(match_data['home_team_goal'] > match_data['away_team_goal']) & (match_data['season'] == '2015/2016')])

In [94]:
arsenal_away = len(match_data[(match_data['away_team_api_id'] == id_arsenal) & (match_data['away_team_goal'] > match_data['home_team_goal'])& (match_data['season'] == '2015/2016')])

In [95]:
arsenal_won_matches_2015_2016 = arsenal_home + arsenal_away # Your code here

In [96]:
arsenal_won_matches_2015_2016

20

In [97]:
assert(isinstance(arsenal_won_matches_2015_2016, int))

**Task 14 (2 points).** Find a team with the highest win rate in the **2015/2016** season. Win rate means won matches / all matches. If there are several teams with the highest win rate return the first by name in lexical order

In [98]:
matches_2015_2016 = match_data[match_data['season'] == '2015/2016']

In [99]:
matches_2015_2016 = matches_2015_2016[['home_team_api_id','away_team_api_id','home_team_goal','away_team_goal']]

In [100]:
matches_2015_2016.head()

,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal
1488,9997,8342,2,1
1489,8571,9985,2,1
1490,9987,1773,3,1
1491,8573,8203,3,1
1492,10000,9994,3,1


In [103]:
all_matches = len(matches_2015_2016)

In [107]:
conditions = [
    (matches_2015_2016['away_team_goal'] > matches_2015_2016['home_team_goal']),
    (matches_2015_2016['away_team_goal'] < matches_2015_2016['home_team_goal']),
    (matches_2015_2016['away_team_goal'] == matches_2015_2016['home_team_goal'])]
choices = [matches_2015_2016['away_team_api_id'], matches_2015_2016['home_team_api_id'], 'None']
matches_2015_2016['won_team'] = np.select(conditions, choices, default='Empty')

In [109]:
matches_2015_2016.head()

,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,won_team
1488,9997,8342,2,1,9997
1489,8571,9985,2,1,8571
1490,9987,1773,3,1,9987
1491,8573,8203,3,1,8573
1492,10000,9994,3,1,10000


In [114]:
d = team_data.astype(str).set_index('team_api_id')['team_long_name']
matches_2015_2016['won_team_name'] = (matches_2015_2016['won_team'].map(d))

In [115]:
matches_2015_2016.head()

,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,won_team,won_team_name
1488,9997,8342,2,1,9997,Sint-Truidense VV
1489,8571,9985,2,1,8571,KV Kortrijk
1490,9987,1773,3,1,9987,KRC Genk
1491,8573,8203,3,1,8573,KV Oostende
1492,10000,9994,3,1,10000,SV Zulte-Waregem


In [128]:
team_highest_winrate_2015_2016 = matches_2015_2016['won_team_name'].value_counts().idxmax()# Your code here

In [129]:
team_highest_winrate_2015_2016

'Paris Saint-Germain'

In [130]:
assert(isinstance(team_highest_winrate_2015_2016, str))

**Task 15 (2 points).** Determine the team with the maximum days' gap between matches in **England Premier League 2010/2011 season**. Return number of days in that gap. <br>
**Note**: a *gap* means the number of days between two consecutive matches of the same team.

In [134]:
league_data.head()

,id,country_id,name
0,1,1,Belgium Jupiler League
1,1729,1729,England Premier League
2,4769,4769,France Ligue 1
3,7809,7809,Germany 1. Bundesliga
4,10257,10257,Italy Serie A


In [135]:
id_eng = league_data[league_data['name'] == 'England Premier League']['id'].values[0]

In [136]:
id_eng

1729

In [187]:
matches_2010_2011 = match_data[(match_data['league_id'] == id_eng) & (match_data['season'] == '2010/2011')]

In [188]:
matches_2010_2011.head()

,id,country_id,league_id,season,stage,date,match_api_id,home_team_api_id,away_team_api_id,home_team_goal,away_team_goal,home_player_X1,home_player_X2,home_player_X3,home_player_X4,home_player_X5,home_player_X6,home_player_X7,home_player_X8,home_player_X9,home_player_X10,home_player_X11,away_player_X1,away_player_X2,away_player_X3,away_player_X4,away_player_X5,away_player_X6,away_player_X7,away_player_X8,away_player_X9,away_player_X10,away_player_X11,home_player_Y1,home_player_Y2,home_player_Y3,home_player_Y4,home_player_Y5,home_player_Y6,home_player_Y7,home_player_Y8,home_player_Y9,home_player_Y10,home_player_Y11,away_player_Y1,away_player_Y2,away_player_Y3,away_player_Y4,away_player_Y5,away_player_Y6,away_player_Y7,away_player_Y8,away_player_Y9,away_player_Y10,away_player_Y11,home_player_1,home_player_2,home_player_3,home_player_4,home_player_5,home_player_6,home_player_7,home_player_8,home_player_9,home_player_10,home_player_11,away_player_1,away_player_2,away_player_3,away_player_4,away_player_5,away_player_6,away_player_7,away_player_8,away_player_9,away_player_10,away_player_11,goal,shoton,shotoff,foulcommit,card,cross,corner,possession,B365H,B365D,B365A,BWH,BWD,BWA,IWH,IWD,IWA,LBH,LBD,LBA,PSH,PSD,PSA,WHH,WHD,WHA,SJH,SJD,SJA,VCH,VCD,VCA,GBH,GBD,GBA,BSH,BSD,BSA
2488,2489,1729,1729,2010/2011,1,2010-08-14 00:00:00,839796,10252,8654,3,0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,5.0,1.0,2.0,4.0,6.0,8.0,1.0,3.0,5.0,7.0,9.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,9.0,11.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,7.0,11.0,30380.0,30357.0,161414.0,24211.0,24136.0,139671.0,30892.0,38609.0,38807.0,23354.0,26165.0,36374.0,35110.0,109897.0,23818.0,26348.0,36394.0,24223.0,37169.0,34590.0,30734.0,34543.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><corners>1</corners></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>67</comment><event...,2.00,3.30,4.00,1.85,3.40,4.10,1.80,3.4,4.0,1.83,3.4,4.33,NaN,NaN,NaN,1.91,3.25,4.33,2.00,3.40,3.75,2.00,3.25,4.20,2.00,3.25,3.75,1.80,3.40,4.33
2489,2490,1729,1729,2010/2011,1,2010-08-14 00:00:00,839797,8655,8668,1,0,1.0,2.0,4.0,6.0,8.0,1.0,3.0,5.0,7.0,9.0,5.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,7.0,11.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,9.0,11.0,30622.0,30658.0,19020.0,23921.0,30739.0,23927.0,186137.0,30342.0,23916.0,25667.0,21613.0,31465.0,30371.0,23268.0,24216.0,24846.0,24006.0,39618.0,24011.0,36012.0,30876.0,30735.0,<goal><value><comment>n</comment><stats><goals...,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>45</comment><event...,2.88,3.25,2.50,2.75,3.25,2.45,2.55,3.2,2.5,2.80,3.3,2.50,NaN,NaN,NaN,2.88,3.20,2.50,2.75,3.30,2.50,3.10,3.30,2.38,2.90,3.20,2.40,2.70,3.25,2.50
2490,2491,1729,1729,2010/2011,1,2010-08-14 00:00:00,839799,8559,9879,0,0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,8.0,6.0,4.0,6.0,1.0,2.0,4.0,6.0,8.0,2.0,4.0,6.0,8.0,5.0,5.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,10.0,10.0,1.0,3.0,3.0,3.0,3.0,7.0,7.0,7.0,7.0,9.0,11.0,23932.0,26454.0,23783.0,40128.0,24728.0,130670.0,35532.0,33633.0,24455.0,34261.0,23934.0,35477.0,37266.0,23780.0,26777.0,33045.0,24020.0,25253.0,30338.0,34574.0,24737.0,24741.0,<goal />,<shoton><value><stats><shoton>1</shoton></stat...,<shotoff><value><stats><shotoff>1</shotoff></s...,<foulcommit><value><stats><foulscommitted>1</f...,<card><value><comment>y</comment><stats><ycard...,<cross><value><stats><crosses>1</crosses></sta...,<corner><value><stats><corners>1</corners></st...,<possession><value><comment>5

In [190]:
matches_2010_2011_1 = matches_2010_2011[['date', 'home_team_api_id']]
matches_2010_2011_1 = matches_2010_2011_1.rename(columns={"home_team_api_id": "team_api_id"})

In [191]:
matches_2010_2011_2 = matches_2010_2011[['date', 'away_team_api_id']]
matches_2010_2011_2 = matches_2010_2011_2.rename(columns={"away_team_api_id": "team_api_id"})

In [192]:
frames = [matches_2010_2011_1, matches_2010_2011_2]
matches_2010_2011 = pd.concat(frames)

In [193]:
matches_2010_2011.head()

,date,team_api_id
2488,2010-08-14 00:00:00,10252
2489,2010-08-14 00:00:00,8655
2490,2010-08-14 00:00:00,8559
2491,2010-08-14 00:00:00,8455
2492,2010-08-15 00:00:00,8650


In [220]:
matches_2010_2011 = matches_2010_2011.sort_values(by=['team_api_id','date'])

In [221]:
matches_2010_2011.head()

,index,date,team_api_id
24,2491,2010-08-14,8455
28,2607,2010-08-21,8455
16,2717,2010-08-28,8455
6,2809,2010-09-11,8455
2,2827,2010-09-19,8455


In [222]:
matches_2010_2011.date = pd.to_datetime(matches_2010_2011.date)

In [223]:
matches_2010_2011 = matches_2010_2011.reset_index()

In [224]:
matches_2010_2011.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 760 entries, 0 to 759
Data columns (total 4 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   level_0      760 non-null    int64         
 1   index        760 non-null    int64         
 2   date         760 non-null    datetime64[ns]
 3   team_api_id  760 non-null    int64         
dtypes: datetime64[ns](1), int64(3)
memory usage: 23.9 KB


In [229]:
highest_gap_england_2010_2011 = matches_2010_2011.groupby(['team_api_id']).diff().abs().max().date.days# Your code here

In [230]:
highest_gap_england_2010_2011

18

In [231]:
assert(isinstance(highest_gap_england_2010_2011, int))

### Warning! Do not change anything in the area below

In [232]:
with open('student_answers.txt', 'w') as file:
    file.write(f"{players_180_190}\n")
    file.write(f"{players_1980}\n")
    file.write(f"{highest_players}\n")
    file.write(f"{years_born_players}\n")
    file.write(f"{round(adriano_mean, 3)} {round(adriano_std, 3)}\n")
    file.write(f"{dow_with_min_players_born}\n")
    file.write(f"{league_most_matches}\n")
    file.write(f"{max_matches_player}\n")
    file.write(f"{';'.join(['%s,%s' % tup for tup in top_correlated_features])};\n")
    file.write(f"{neymar_similarities}\n")
    file.write(f"{borussia_bundesliga_2008_2009_matches}\n")
    file.write(f"{team_most_matches_bundesliga_2008_2009}\n")
    file.write(f"{arsenal_won_matches_2015_2016}\n")
    file.write(f"{team_highest_winrate_2015_2016}\n")
    file.write(f"{highest_gap_england_2010_2011}\n")